In [12]:
from pyspark.sql.functions import col, trim, upper, when, lit, current_timestamp, \
                                  substring, regexp_replace, unix_timestamp, \
                                  year, month, dayofmonth, dayofweek, avg, min, max, \
                                  floor, hour, date_format, desc, lag, abs, \
                                  lead, datediff, when, window, mean, stddev, skewness, kurtosis, \
                                  sum, to_date, count, concat, countDistinct \

from pyspark.sql.window import Window
from pyspark.ml.feature import QuantileDiscretizer

from pyspark.sql.types import StringType, IntegerType, DoubleType, LongType, TimestampType

import plotly.express as px
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import functions as F


height = 600

bronze_path = "Files/bronze/transacciones_servicios/"
silver_path = "Files/silver/transacciones_servicios/"

notebook_version = "1.0"

print(f"Ruta Bronze: {bronze_path}")
print(f"Ruta Silver: {silver_path}")

StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 14, Finished, Available, Finished)

Ruta Bronze: Files/bronze/transacciones_servicios/
Ruta Silver: Files/silver/transacciones_servicios/


In [13]:
#cargamos los datos de transacciones a terceros 
df_transacciones_servicios = spark.read.format('delta')\
                                      .load(silver_path)\

print(f"Lectura completada. Se leyeron {df_transacciones_servicios.count()} registros.")
df_transacciones_servicios.printSchema()

StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 15, Finished, Available, Finished)

Lectura completada. Se leyeron 9474509 registros.
root
 |-- account_nbr: long (nullable = true)
 |-- ident: string (nullable = true)
 |-- posting_date: string (nullable = true)
 |-- product_type_id: integer (nullable = true)
 |-- description1: string (nullable = true)
 |-- description2: string (nullable = true)
 |-- description3: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- cashier_nbr: integer (nullable = true)
 |-- to_account_nbr: long (nullable = true)
 |-- idsegmento: integer (nullable = true)
 |-- anio_transaccion: integer (nullable = true)
 |-- mes_transaccion: integer (nullable = true)
 |-- dia_transaccion: integer (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- fecha_proceso_silver: timestamp (nullable = true)
 |-- version_notebook: string (nullable = true)
 |-- nombre_dia_semana: string (nullable = true)
 |-- monto_anterior: double (nullable = true)
 |-- fecha_anterior: string (nullable = true)
 |-- fecha_siguiente_transaccion: string (

In [14]:
# calculamos la cantidad de dias de diferencia entre las transaccioes
df_featured = df_transacciones_servicios.withColumn("dias_diferencia_anterior_transaccion", datediff(col("posting_date"), col("fecha_anterior"))) \
                         .withColumn("diferencia_monto_anterior", abs(col("amount") - col("monto_anterior")))\
                         .withColumn("dias_diferencia_siguiente_transaccion",datediff(col("fecha_siguiente_transaccion"), col("posting_date")))\
                         .withColumn("diferencia_monto_siguiente", abs(col("amount") - col("monto_siguiente_transaccion")))

StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 16, Finished, Available, Finished)

In [15]:
"""
vamos a agrupar los dias de diferencia de manera descriptiva para poder realizar un proceso de interpretación 
"""

df_featured = df_featured.withColumn("grupo_dias_diferencia_anterior",
                                    when(col("dias_diferencia_anterior_transaccion").isNull(), "Primera Transacción")
                                    .when(col("dias_diferencia_anterior_transaccion") == 0, "Mismo Día")
                                    .when(col("dias_diferencia_anterior_transaccion").between(1, 7), "Semanal (1-7 días)")
                                    .when(col("dias_diferencia_anterior_transaccion").between(8, 30), "Mensual (8-30 días)")
                                    .when(col("dias_diferencia_anterior_transaccion").between(31, 90), "Trimestral (31-90 días)")
                                    .otherwise("Largo Plazo (>90 días)"))

df_featured = df_featured.withColumn("grupo_dias_diferencia_siguiente",
                                    when(col("dias_diferencia_siguiente_transaccion").isNull(), "Primera Transacción")
                                    .when(col("dias_diferencia_siguiente_transaccion") == 0, "Mismo Día")
                                    .when(col("dias_diferencia_siguiente_transaccion").between(1, 7), "Semanal (1-7 días)")
                                    .when(col("dias_diferencia_siguiente_transaccion").between(8, 30), "Mensual (8-30 días)")
                                    .when(col("dias_diferencia_siguiente_transaccion").between(31, 90), "Trimestral (31-90 días)")
                                    .otherwise("Largo Plazo (>90 días)"))

StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 17, Finished, Available, Finished)

In [16]:
df_featured = df_featured.select(
    col("account_nbr").alias('numero_cuenta_remitente'),
    col("ident").alias('cedula_remitente'),
    col("posting_date").alias('fecha_transaccion'),
    "product_type_id",
    "description1",
    "description2",
    "description3",
    col("amount").alias('monto'),
    "cashier_nbr",
    col("to_account_nbr").alias('numero_cuenta_servicio'),
    col("idsegmento").alias('segmento'),
    "anio_transaccion",
    "mes_transaccion",
    "dia_transaccion",
    "dia_semana",
    "nombre_dia_semana",
    "monto_anterior",
    "fecha_anterior",
    "fecha_siguiente_transaccion",
    "monto_siguiente_transaccion",
    "dias_diferencia_anterior_transaccion",
    "grupo_dias_diferencia_anterior",
    "diferencia_monto_anterior",
    "dias_diferencia_siguiente_transaccion",
    "grupo_dias_diferencia_siguiente", 
    "diferencia_monto_siguiente",
    "fecha_proceso_silver",
    "version_notebook"
)


StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 18, Finished, Available, Finished)

In [17]:
display(df_featured)

StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7947d150-59c7-41d7-a8a9-a7e6fc94f503)

In [18]:
from pyspark.sql.functions import mean, stddev, skewness, kurtosis

# Filtrar por description1 que contiene "DGII"
df_dgii = df_featured.filter(col("description1").like("%DGII%"))

# Calcular percentiles de monto solo para esas filas
percentiles = df_dgii.approxQuantile("monto", [0.01, 0.25, 0.5, 0.75, 0.99], 0.001)

# Calcular otras estadísticas
stats = df_dgii.select(
    mean("monto").alias("media"),
    stddev("monto").alias("desv_estandar"),
    skewness("monto").alias("asimetria"),
    kurtosis("monto").alias("curtosis")
).first()

print("--- Perfil Estadístico del Monto (filtrado por DGII) ---")
print(f"Media: {stats['media']:,.2f}")
print(f"Desviación Estándar: {stats['desv_estandar']:,.2f}")
print(f"Asimetría: {stats['asimetria']:.2f}")
print(f"Curtosis: {stats['curtosis']:.2f}")
print("\n--- Percentiles ---")
print(f"1% (Percentil 1): {percentiles[0]:,.2f}")
print(f"25% (Cuartil 1): {percentiles[1]:,.2f}")
print(f"50% (Mediana): {percentiles[2]:,.2f}")
print(f"75% (Cuartil 3): {percentiles[3]:,.2f}")
print(f"99% (Percentil 99): {percentiles[4]:,.2f}")


StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 20, Finished, Available, Finished)

--- Perfil Estadístico del Monto (filtrado por DGII) ---
Media: 10,003.33
Desviación Estándar: 54,144.83
Asimetría: 43.10
Curtosis: 4455.44

--- Percentiles ---
1% (Percentil 1): 80.00
25% (Cuartil 1): 80.00
50% (Mediana): 80.00
75% (Cuartil 3): 2,077.25
99% (Percentil 99): 174,694.05


In [19]:
from pyspark.sql.functions import mean, stddev, skewness, kurtosis

# Filtrar por description1 que contiene "DGA"
df_dga = df_featured.filter(col("description1").like("%DGA%"))

# Calcular percentiles de monto solo para esas filas
percentiles = df_dga.approxQuantile("monto", [0.01, 0.25, 0.5, 0.75, 0.99], 0.001)

# Calcular otras estadísticas
stats = df_dga.select(
    mean("monto").alias("media"),
    stddev("monto").alias("desv_estandar"),
    skewness("monto").alias("asimetria"),
    kurtosis("monto").alias("curtosis")
).first()

print("--- Perfil Estadístico del Monto (filtrado por DGA) ---")
print(f"Media: {stats['media']:,.2f}")
print(f"Desviación Estándar: {stats['desv_estandar']:,.2f}")
print(f"Asimetría: {stats['asimetria']:.2f}")
print(f"Curtosis: {stats['curtosis']:.2f}")
print("\n--- Percentiles ---")
print(f"1% (Percentil 1): {percentiles[0]:,.2f}")
print(f"25% (Cuartil 1): {percentiles[1]:,.2f}")
print(f"50% (Mediana): {percentiles[2]:,.2f}")
print(f"75% (Cuartil 3): {percentiles[3]:,.2f}")
print(f"99% (Percentil 99): {percentiles[4]:,.2f}")


StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 21, Finished, Available, Finished)

--- Perfil Estadístico del Monto (filtrado por DGA) ---
Media: 67,901.92
Desviación Estándar: 206,109.96
Asimetría: 11.30
Curtosis: 237.70

--- Percentiles ---
1% (Percentil 1): 299.95
25% (Cuartil 1): 500.00
50% (Mediana): 2,618.61
75% (Cuartil 3): 70,141.88
99% (Percentil 99): 820,412.95


In [20]:
from pyspark.sql.functions import mean, stddev, skewness, kurtosis

# Filtrar por description1 que contiene "TSS"
df_tss = df_featured.filter(col("description1").like("%TSS%"))

# Calcular percentiles de monto solo para esas filas
percentiles = df_tss.approxQuantile("monto", [0.01, 0.25, 0.5, 0.75, 0.99], 0.001)

# Calcular otras estadísticas
stats = df_tss.select(
    mean("monto").alias("media"),
    stddev("monto").alias("desv_estandar"),
    skewness("monto").alias("asimetria"),
    kurtosis("monto").alias("curtosis")
).first()

print("--- Perfil Estadístico del Monto (filtrado por TSS) ---")
print(f"Media: {stats['media']:,.2f}")
print(f"Desviación Estándar: {stats['desv_estandar']:,.2f}")
print(f"Asimetría: {stats['asimetria']:.2f}")
print(f"Curtosis: {stats['curtosis']:.2f}")
print("\n--- Percentiles ---")
print(f"1% (Percentil 1): {percentiles[0]:,.2f}")
print(f"25% (Cuartil 1): {percentiles[1]:,.2f}")
print(f"50% (Mediana): {percentiles[2]:,.2f}")
print(f"75% (Cuartil 3): {percentiles[3]:,.2f}")
print(f"99% (Percentil 99): {percentiles[4]:,.2f}")


StatementMeta(, 5bfff979-3240-49a9-9b59-04e7ea92d74a, 22, Finished, Available, Finished)

--- Perfil Estadístico del Monto (filtrado por TSS) ---
Media: 15,495.49
Desviación Estándar: 48,624.61
Asimetría: 116.59
Curtosis: 27164.84

--- Percentiles ---
1% (Percentil 1): 2,528.75
25% (Cuartil 1): 3,445.15
50% (Mediana): 8,440.22
75% (Cuartil 3): 15,362.16
99% (Percentil 99): 113,883.10
